In [50]:
import pandas as pd
from dotenv import load_dotenv
import os
from api import get_token, get_camera_token
from pyroclient import Client
import glob
from PIL import Image
import numpy as np
import io
import requests
import random
import shutil
from utils import read_pred_file
import io
import requests
from PIL import Image
import time
import pandas as pd
import requests
from PIL import Image
import io
import time

In [51]:
API_URL = "http://0.0.0.0:5050"
load_dotenv("../.env")
SUPERADMIN_LOGIN = os.environ.get("SUPERADMIN_LOGIN")
SUPERADMIN_PWD = os.environ.get("SUPERADMIN_PWD")

# Get access token
admin_access_token = get_token(API_URL, SUPERADMIN_LOGIN, SUPERADMIN_PWD)


In [52]:
users = pd.read_csv("../data/csv/API_DATA_DEV - users.csv")
cameras = pd.read_csv("../data/csv/API_DATA_DEV - cameras.csv")
users

id  organization_id password    login   role                  created_at
0   2                2  @test77  sdis-77  agent  2024-02-23T08:18:45.447773
1   3                3     test   test07  agent  2024-02-23T08:18:45.447773

In [53]:
user = "sdis-77"
organization_id = users[users["login"]==user]["organization_id"].item()

user_cameras = cameras[cameras["organization_id"]==organization_id]
user_cameras

id  organization_id                name  angle_of_view  elevation      lat  \
0   2                2         videlles-01           54.2        110  48.4783   
1   3                2         videlles-02           54.2        110  48.4783   
2   4                2      croix-augas-01           54.2        110  48.4267   
3   5                2      croix-augas-02           54.2        110  48.4267   
4   6                2  moret-sur-loing-01           54.2        110  48.3792   
5   7                2  moret-sur-loing-02           54.2        110  48.3792   
6   8                2          nemours-01           54.2        110  48.2605   
7   9                2          nemours-02           54.2        110  48.2605   

      lon  is_trustable  last_active_at                  created_at  
0  2.4242          True             NaN  2023-11-07T15:07:19.226673  
1  2.4242          True             NaN  2023-11-07T15:07:19.226673  
2  2.7109          True             NaN  2023-11-07T15:07:19.226673  
3  2.7109          True             NaN  2023-11-07T15:07:19.226673  
4  2.8208          True             NaN  2023-11-07T15:07:19.226673  
5  2.8208          True             NaN  2023-11-07T15:07:19.226673  
6  2.7064          True             NaN  2023-11-07T15:07:19.226673  
7  2.7064          True             NaN  2023-11-07T15:07:19.226673

In [54]:
# Load both DataFrames
df4 = pd.read_json("../data/demo77/croix-augas_-_190-13-24/response.json")
df5 = pd.read_json("../data/demo77/moret-sur-loing_-_280-13-22/response.json")
df8 = pd.read_json("../data/demo77/nemours_-_25-13-24/response.json")

# Initialize camera IDs and azimuths
cam4_id = 4
cam5_id = 5
cam8_id = 8
cam4_azimuth = df4["azimuth"].iloc[0]
cam5_azimuth = df5["azimuth"].iloc[0]
cam8_azimuth = df8["azimuth"].iloc[0]

# Get clients
client4 = Client(get_camera_token(API_URL, cam4_id, admin_access_token), API_URL)
client5 = Client(get_camera_token(API_URL, cam5_id, admin_access_token), API_URL)
client8 = Client(get_camera_token(API_URL, cam8_id, admin_access_token), API_URL)

# Create iterators
iter4 = df4.iterrows()
iter5 = df5.iterrows()
iter8 = df8.iterrows()

loop_count = 0

In [55]:
import time
import requests
import io
from PIL import Image

def send_detection(row, client, azimuth, cam_label):
    try:
        response = requests.get(row["url"])
        response.raise_for_status()
        im = Image.open(io.BytesIO(response.content))
        stream = io.BytesIO()
        im.save(stream, format="JPEG", quality=100)
        bboxes = row["bboxes"]
        if isinstance(bboxes, str):
            bboxes = eval(bboxes)
        response = client.create_detection(stream.getvalue(), azimuth, bboxes)
        print(f"[{cam_label}] Created detection with ID: {response.json()['id']}")
        return True
    except Exception as e:
        print(f"[{cam_label}] Error: {e}")
        return False

# Step 1: Send 10 from Camera 4
for _ in range(10):
    try:
        idx4, row4 = next(iter4)
        send_detection(row4, client4, cam4_azimuth, "Cam4")
    except StopIteration:
        print("Camera 4: No more detections.")
        break

# Step 2: Sleep
time.sleep(20)

# Step 6: Send 10 from Camera 8
for _ in range(10):
    try:
        idx8, row8 = next(iter8)
        send_detection(row8, client8, cam8_azimuth, "Cam8")
    except StopIteration:
        print("Camera 8: No more detections.")
        break



# Step 4: Send 1 more from Camera 4
try:
    idx4, row4 = next(iter4)
    send_detection(row4, client4, cam4_azimuth, "Cam4")
except StopIteration:
    print("Camera 4: No more detections.")

# Step 5: Sleep
time.sleep(20)

# Step 3: Send 10 from Camera 5
for _ in range(10):
    try:
        idx5, row5 = next(iter5)
        send_detection(row5, client5, cam5_azimuth, "Cam5")
    except StopIteration:
        print("Camera 5: No more detections.")
        break

# Step 7: Send all remaining from all
while True:
    active = False
    try:
        idx4, row4 = next(iter4)
        send_detection(row4, client4, cam4_azimuth, "Cam4")
        active = True
    except StopIteration:
        pass
    try:
        idx5, row5 = next(iter5)
        send_detection(row5, client5, cam5_azimuth, "Cam5")
        active = True
    except StopIteration:
        pass
    try:
        idx8, row8 = next(iter8)
        send_detection(row8, client8, cam8_azimuth, "Cam8")
        active = True
    except StopIteration:
        pass
    if not active:
        print("All cameras: No more detections.")
        break

    time.sleep(30)


[Cam4] Created detection with ID: 1
[Cam4] Created detection with ID: 2
[Cam4] Created detection with ID: 3
[Cam4] Created detection with ID: 4
[Cam4] Created detection with ID: 5
[Cam4] Created detection with ID: 6
[Cam4] Created detection with ID: 7
[Cam4] Created detection with ID: 8
[Cam4] Created detection with ID: 9
[Cam4] Created detection with ID: 10
[Cam8] Error: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
[Cam8] Created detection with ID: 11
[Cam8] Created detection with ID: 12
[Cam8] Created detection with ID: 13
[Cam8] Created detection with ID: 14
[Cam8] Created detection with ID: 15
[Cam8] Created detection with ID: 16
[Cam8] Created detection with ID: 17
[Cam8] Created detection with ID: 18
[Cam8] Created detection with ID: 19
Camera 4: No more detections.
[Cam5] Created detection with ID: 20
[Cam5] Created detection with ID: 21
[Cam5] Created detection with ID: 22
[Cam5] Created detection with ID: 23
[Cam5] Created detection with ID: 24

In [56]:
ddd= ddd54

NameError: name 'ddd54' is not defined

In [ ]:
for i in range(10):
    try:
        # Always send from camera 4
        idx4, row4 = next(iter4)
        response = requests.get(row4["url"])
        response.raise_for_status()
        im = Image.open(io.BytesIO(response.content))
        stream = io.BytesIO()
        im.save(stream, format="JPEG", quality=100)
        bboxes = row4["bboxes"]
        if isinstance(bboxes, str):
            bboxes = eval(bboxes)
        response = client4.create_detection(stream.getvalue(), cam4_azimuth, bboxes)
        print(f"[Cam4] Created detection with ID: {response.json()['id']}")
    except StopIteration:
        print("Camera 4: No more detections.")
        break

time.sleep(60)

for i in range(10):

    try:
        idx5, row5 = next(iter5)
        response = requests.get(row5["url"])
        response.raise_for_status()
        im = Image.open(io.BytesIO(response.content))
        stream = io.BytesIO()
        im.save(stream, format="JPEG", quality=100)
        bboxes = row5["bboxes"]
        if isinstance(bboxes, str):
            bboxes = eval(bboxes)
        response = client5.create_detection(stream.getvalue(), cam5_azimuth, bboxes)
        print(f"[Cam5] Created detection with ID: {response.json()['id']}")
    except StopIteration:
        print("Camera 5: No more detections.")


time.sleep(60)

for i in range(10):
    try:
        idx8, row8 = next(iter8)
        response = requests.get(row8["url"])
        response.raise_for_status()
        im = Image.open(io.BytesIO(response.content))
        stream = io.BytesIO()
        im.save(stream, format="JPEG", quality=100)
        bboxes = row8["bboxes"]
        if isinstance(bboxes, str):
            bboxes = eval(bboxes)
        response = client8.create_detection(stream.getvalue(), cam8_azimuth, bboxes)
        print(f"[Cam7] Created detection with ID: {response.json()['id']}")
    except StopIteration:
        print("Camera 7: No more detections.")



In [ ]:
loop_count = 0

while True:
    try:
        # Always send from camera 4
        idx4, row4 = next(iter4)
        response = requests.get(row4["url"])
        response.raise_for_status()
        im = Image.open(io.BytesIO(response.content))
        stream = io.BytesIO()
        im.save(stream, format="JPEG", quality=100)
        bboxes = row4["bboxes"]
        if isinstance(bboxes, str):
            bboxes = eval(bboxes)
        response = client4.create_detection(stream.getvalue(), cam4_azimuth, bboxes)
        print(f"[Cam4] Created detection with ID: {response.json()['id']}")
    except StopIteration:
        print("Camera 4: No more detections.")
        break

  
    if loop_count > 3:
        try:
            idx5, row5 = next(iter5)
            response = requests.get(row5["url"])
            response.raise_for_status()
            im = Image.open(io.BytesIO(response.content))
            stream = io.BytesIO()
            im.save(stream, format="JPEG", quality=100)
            bboxes = row5["bboxes"]
            if isinstance(bboxes, str):
                bboxes = eval(bboxes)
            response = client5.create_detection(stream.getvalue(), cam5_azimuth, bboxes)
            print(f"[Cam5] Created detection with ID: {response.json()['id']}")
        except StopIteration:
            print("Camera 5: No more detections.")


    if loop_count > 9:
        try:
            idx8, row8 = next(iter8)
            response = requests.get(row8["url"])
            response.raise_for_status()
            im = Image.open(io.BytesIO(response.content))
            stream = io.BytesIO()
            im.save(stream, format="JPEG", quality=100)
            bboxes = row8["bboxes"]
            if isinstance(bboxes, str):
                bboxes = eval(bboxes)
            response = client8.create_detection(stream.getvalue(), cam8_azimuth, bboxes)
            print(f"[Cam7] Created detection with ID: {response.json()['id']}")
        except StopIteration:
            print("Camera 7: No more detections.")

    loop_count += 1
    #time.sleep(1)


[Cam4] Created detection with ID: 1
[Cam4] Created detection with ID: 2
[Cam4] Created detection with ID: 3
[Cam4] Created detection with ID: 4
[Cam4] Created detection with ID: 5
[Cam5] Created detection with ID: 6
[Cam4] Created detection with ID: 7
[Cam5] Created detection with ID: 8
[Cam4] Created detection with ID: 9
[Cam5] Created detection with ID: 10
[Cam4] Created detection with ID: 11
[Cam5] Created detection with ID: 12
[Cam4] Created detection with ID: 13
[Cam5] Created detection with ID: 14
[Cam4] Created detection with ID: 15
[Cam5] Created detection with ID: 16
[Cam4] Created detection with ID: 17
[Cam5] Created detection with ID: 18
[Cam7] Created detection with ID: 19
[Cam4] Created detection with ID: 20
[Cam5] Created detection with ID: 21
[Cam7] Created detection with ID: 22
[Cam4] Created detection with ID: 23
[Cam5] Created detection with ID: 24
[Cam7] Created detection with ID: 25
[Cam4] Created detection with ID: 26
[Cam5] Created detection with ID: 27
[Cam7] Cre

In [ ]:
# Croix Augas
camera_id = 4

df = pd.read_json("../data/demo77/croix-augas_-_190-13-24/response_1753376057846.json")
df.head()



id  camera_id  sequence_id  azimuth                      bucket_key  \
0  436421         41        10538      190  41-20250714112431-d1a16c83.jpg   
1  436422         41        10538      190  41-20250714112432-091153ee.jpg   
2  436423         41        10538      190  41-20250714112433-b966b57a.jpg   
3  436424         41        10538      190  41-20250714112434-bc27e795.jpg   
4  436425         41        10538      190  41-20250714112434-e7a60517.jpg   

                          bboxes                 created_at  \
0  [(0,0.530,0.018,0.553,0.183)] 2025-07-14 11:24:31.405329   
1  [(0,0.530,0.018,0.553,0.183)] 2025-07-14 11:24:32.336345   
2  [(0,0.530,0.018,0.553,0.183)] 2025-07-14 11:24:33.297084   
3  [(0,0.530,0.018,0.553,0.183)] 2025-07-14 11:24:34.244964   
4  [(0,0.530,0.018,0.553,0.183)] 2025-07-14 11:24:35.011390   

                                                 url  
0  https://s3.sbg.io.cloud.ovh.net/ovh-alert-api-...  
1  https://s3.sbg.io.cloud.ovh.net/ovh-alert-api-...  
2  https://s3.sbg.io.cloud.ovh.net/ovh-alert-api-...  
3  https://s3.sbg.io.cloud.ovh.net/ovh-alert-api-...  
4  https://s3.sbg.io.cloud.ovh.net/ovh-alert-api-...